In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
import molpack as mpk

In [2]:
H2O = mp.Struct(
    "h2o",
    atoms=[
        mp.Atom(name="O", x=0.00000, y=-0.06556, z=0.00000),
        mp.Atom(name="H1", x=0.75695, y=0.52032, z=0.00000),
        mp.Atom(name="H2", x=-0.75695, y=0.52032, z=0.00000),
    ],
    bonds=[],
)
H2O.add_bond_("O", "H1")
H2O.add_bond_("O", "H2")

<Struct h2o: 3 atoms>

In [6]:
water_box = mpk.Target(
    H2O.to_frame(),
    number = 10,
    region = mp.region.Cuboid([0, 0, 0], [3, 4, 5]),
    name="water_box"
)
water_box

<Target water_box: density=0.33>

In [74]:
packer = mpk.NloptPacker()
packer.add_target(water_box)
xyz = packer.pack()

Using COBYLA (Constrained Optimization BY Linear Approximations) (local, no-derivative) algorithm
Number of to be optimized parameters: 90


invalid_argument: 

In [ ]:
import nlopt
import numpy as np

# 目标函数：最大化原子间距
def objective_function(x, grad):
    xyz = x.reshape(-1, 3)
    # 计算所有原子间的距离
    distances = np.linalg.norm(xyz[:, None, :] - xyz[None, :, :], axis=-1)
    np.fill_diagonal(distances, np.inf)  # 忽略自距离
    min_distance = distances.min()  # 找到最小距离
    if grad.size > 0:
        raise NotImplementedError("Gradient computation is not implemented")
    return -min_distance  # 目标是最大化最小距离，因此取负号

# 几何约束：限制所有点在上下界范围内
def geometry_constraint(result, x, grad):
    xyz = x.reshape(-1, 3)
    upper_violation = xyz - np.array([3, 4, 5])
    lower_violation = np.zeros(3, ) - xyz
    # 计算约束值（最大违约情况）
    violations = np.maximum(upper_violation, 0).max() + np.maximum(lower_violation, 0).max()
    result[0] = violations
    if grad.size > 0:
        raise NotImplementedError("Gradient computation is not implemented")

# 初始化内部优化器
inner_opt = nlopt.opt(nlopt.LD_MMA, 90)  # 使用 LD_MMA 优化器
inner_opt.set_min_objective(objective_function)

# 设置边界
# inner_opt.set_lower_bounds(lower_bounds)
# inner_opt.set_upper_bounds(upper_bounds)

# 初始化 AUGLAG 优化器
opt = nlopt.opt(nlopt.AUGLAG, 90)
opt.set_local_optimizer(inner_opt)

# 添加几何约束
tol = np.array([1e-8])  # 容差
opt.add_inequality_mconstraint(geometry_constraint, tol)

# 设置初始值
x0 = np.zeros(90)

# 开始优化
result = opt.optimize(x0)
optimal_value = opt.last_optimum_value()

print(f"Optimal solution: {result}")
print(f"Optimal value: {optimal_value}")

NameError: name 'num_vars' is not defined

[[1318.0781416475195, 1328.179354045545, 1339.713267558748],
 [1368.8238990076543, -141.16545139246955, -137.47845504647086],
 [-140.57796652364223, -111.43112622188471, -100.54820690993306],
 [-88.26054286682259, -79.98655820671307, -69.78543040467152],
 [-45.36327581443838, -41.166393887893406, -39.18993818755069],
 [-42.926843689866025, -16.882155649288713, -10.10616342329023],
 [-1.61189382359925, 5.074421660940758, 11.548589893015038],
 [30.645642791798696, 33.136303604421556, 32.926495320470416],
 [28.052069928192914, 45.6010277709035, 50.67168836687925],
 [56.91081565999413, 60.03137856367809, 65.13211765849773],
 [79.42996249046895, 78.37221644107818, 75.88766796406426],
 [68.44491743372497, 83.47299637897079, 83.50022001971557],
 [88.75823293311454, 86.19670382031474, 89.65376836697729],
 [98.62597198530645, 95.82947404552536, 91.472651844464],
 [82.06925411967957, 91.95295857113962, 90.55965845040508],
 [89.3031302714254, 87.73450463973366, 87.28106776340915],
 [91.7839043830